In [1]:
import pandas as pd
import numpy as np
import datatable as dt

In [2]:
data=pd.read_csv('/Users/juileebhosale/Documents/Personal/JOB/Study/R Practice/user_invoice_summary_sample.csv')
data.head()

,USER_TOKEN,FIRST_SEND_APPLICATION,BUSINESS_CATEGORY,POSTAL_CODE,COUNTRY_CODE,FIRST_INVOICE_MONTH,COUNT_INVOICES_FIRST7D,COUNT_SENT_INVOICES_FIRST7D,COUNT_PAID_INVOICES_FIRST7D,TOTAL_INVOICE_AMOUNT_CENTS_FIRST7D,COUNT_PAYERS_FIRST7D,HAS_USED_AUTOMATIC_REMINDERS_FIRST7D,AVERAGE_INVOICE_AMOUNT_CENTS_FIRST7D,PCT_INVOICES_PAID_FIRST7D,COUNT_DELIVERY_METHODS_FIRST7D,HAS_ACCEPTED_CREDIT_OR_DEBIT_CARD_FIRST7D,RETAINED_IND
0,0EXR4701P1E6X,register,retail,21212,US,2018-01-01,1,1,1,14400,1,0,1.440000e+04,1.000000,1,1,0
1,S5QV1PB8SYQ72,register,home_and_repair,44077-4621,US,2018-08-01,2,2,1,160200,2,0,8.010000e+04,0.500000,2,1,0
2,9ZTQV2QBRHPGJ,dashboard,food_and_drink,19146,US,2018-04-01,3,3,2,483000,2,0,1.610000e+05,0.666666,1,1,0
3,0Z20BGZZTD238,dashboard,charities_education_and_membership,21202,US,2016-11-01,41,41,32,325417120,41,0,7.937003e+06,0.780487,1,1,1
4,2H0WCPHJ2A0MK,dashboard,health_care_and_fitness,97321-2006,US,2017-03-01,19,18,18,28490035,18,0,1.499476e+06,1.000000,2,1,0


### 01. Data Pre-processing

In [3]:
#Check for null values
data.isna().sum()

USER_TOKEN                                    0
FIRST_SEND_APPLICATION                       48
BUSINESS_CATEGORY                             0
POSTAL_CODE                                   0
COUNTRY_CODE                                  0
FIRST_INVOICE_MONTH                           0
COUNT_INVOICES_FIRST7D                        0
COUNT_SENT_INVOICES_FIRST7D                   0
COUNT_PAID_INVOICES_FIRST7D                   0
TOTAL_INVOICE_AMOUNT_CENTS_FIRST7D            0
COUNT_PAYERS_FIRST7D                          0
HAS_USED_AUTOMATIC_REMINDERS_FIRST7D          0
AVERAGE_INVOICE_AMOUNT_CENTS_FIRST7D          0
PCT_INVOICES_PAID_FIRST7D                     0
COUNT_DELIVERY_METHODS_FIRST7D                0
HAS_ACCEPTED_CREDIT_OR_DEBIT_CARD_FIRST7D     0
RETAINED_IND                                  0
dtype: int64

In [5]:
#Replace NA values
data['FIRST_SEND_APPLICATION'].fillna('No Value',inplace=True)

In [6]:
#% of values
data.groupby('RETAINED_IND').size()/len(data)*100

RETAINED_IND
0    74.473287
1    25.526713
dtype: float64

### 02. Filter into train and test

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19223 entries, 0 to 19222
Data columns (total 17 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   USER_TOKEN                                 19223 non-null  object 
 1   FIRST_SEND_APPLICATION                     19223 non-null  object 
 2   BUSINESS_CATEGORY                          19223 non-null  object 
 3   POSTAL_CODE                                19223 non-null  object 
 4   COUNTRY_CODE                               19223 non-null  object 
 5   FIRST_INVOICE_MONTH                        19223 non-null  object 
 6   COUNT_INVOICES_FIRST7D                     19223 non-null  int64  
 7   COUNT_SENT_INVOICES_FIRST7D                19223 non-null  int64  
 8   COUNT_PAID_INVOICES_FIRST7D                19223 non-null  int64  
 9   TOTAL_INVOICE_AMOUNT_CENTS_FIRST7D         19223 non-null  int64  
 10  COUNT_PAYERS_FIRST7D  

In [9]:
#Filter columns
columns_to_use=['COUNT_INVOICES_FIRST7D','COUNT_SENT_INVOICES_FIRST7D','COUNT_PAID_INVOICES_FIRST7D',
                'TOTAL_INVOICE_AMOUNT_CENTS_FIRST7D','COUNT_PAYERS_FIRST7D','HAS_USED_AUTOMATIC_REMINDERS_FIRST7D',
               'AVERAGE_INVOICE_AMOUNT_CENTS_FIRST7D','PCT_INVOICES_PAID_FIRST7D','COUNT_DELIVERY_METHODS_FIRST7D',
               'HAS_ACCEPTED_CREDIT_OR_DEBIT_CARD_FIRST7D']
X_train=train[columns_to_use]
y_train=train['RETAINED_IND']
X_test=test[columns_to_use]
y_test=test['RETAINED_IND']

In [10]:
from sklearn.linear_model import LogisticRegression
model_1=LogisticRegression()
model_1.fit(X_train,y_train)

LogisticRegression()

In [11]:
y_pred=model_1.predict_proba(X_train)

In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train,y_pred[:,1])

0.847097247158753

In [13]:
y_pred_test=model_1.predict_proba(X_test)

In [14]:
roc_auc_score(y_test,y_pred_test[:,1])

0.8463742535512269

In [ ]:
# Process
# 1. Read data
# 2. Count missing data. Fill missing data 
#     Replace with zero, some value.
#     Also learn replacing with mean/average
# 3. Handle categorical variables
# 3. Train - test split
# 4. Choose variables to use for model
#     You can also use fancier DS processes like lasso or ridge selection
# 5. Model build
#     Experiment with linear/logistic regression, xgboost, random forest
# 6. Make predictions
# 7. Get performance
# 8. Plot performance - bonus


### Testing

In [ ]:
data=pd.read_csv('/Users/juileebhosale/Documents/Personal/JOB/Study/R Practice/user_invoice_summary_sample.csv')
data.head()

In [18]:
data[data.USER_TOKEN=='2F7PYM0BJFG4H']

,USER_TOKEN,FIRST_SEND_APPLICATION,BUSINESS_CATEGORY,POSTAL_CODE,COUNTRY_CODE,FIRST_INVOICE_MONTH,COUNT_INVOICES_FIRST7D,COUNT_SENT_INVOICES_FIRST7D,COUNT_PAID_INVOICES_FIRST7D,TOTAL_INVOICE_AMOUNT_CENTS_FIRST7D,COUNT_PAYERS_FIRST7D,HAS_USED_AUTOMATIC_REMINDERS_FIRST7D,AVERAGE_INVOICE_AMOUNT_CENTS_FIRST7D,PCT_INVOICES_PAID_FIRST7D,COUNT_DELIVERY_METHODS_FIRST7D,HAS_ACCEPTED_CREDIT_OR_DEBIT_CARD_FIRST7D,RETAINED_IND
16750,2F7PYM0BJFG4H,dashboard,retail,36401,US,2018-05-01,2,2,1,54000,2,0,27000.0,0.5,1,1,0


In [17]:
data.loc[data.USER_TOKEN=='2F7PYM0BJFG4H','FIRST_SEND_APPLICATION']='dashboard'